In [1]:
import numpy as np
import torch
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset

In [4]:
class Sum_Dataset(Dataset):
    def __init__(self, data_path, lab_path):
        data = np.load(data_path)
        labels = np.load(lab_path)
        self.images = torch.tensor(data, dtype=torch.float32) / 255.0
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [5]:
class Standard_CNN_model(torch.nn.Module):
    def __init__(self):
        super(Standard_CNN_model, self).__init__()
        self.cnn = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2)
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(64 * 10 * 42, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.cnn(x)
        # print(x.shape)
        x = self.fc(x)
        return x


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

train_dataset = Sum_Dataset('DL-Project/data0.npy', 'DL-Project/lab0.npy')
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = Sum_Dataset('DL-Project/data1.npy', 'DL-Project/lab1.npy')
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

Using device: cuda


In [7]:
model = Standard_CNN_model().to(device)
criterion = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in tqdm(range(10)):
    model.train()
    for images, labels in train_dataloader:
        images, labels = images.unsqueeze(1).to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.unsqueeze(1).to(device), labels.to(device)
            outputs = model(images)
            val_loss += criterion(outputs.squeeze(), labels).item()
    val_loss /= len(val_dataloader)

    print(f'Epoch [{epoch+1}/10], Train Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}')

 10%|█         | 1/10 [00:03<00:32,  3.57s/it]

Epoch [1/10], Train Loss: 33.4099, Val Loss: 30.7896


 20%|██        | 2/10 [00:05<00:19,  2.40s/it]

Epoch [2/10], Train Loss: 33.9657, Val Loss: 25.5134


 30%|███       | 3/10 [00:06<00:14,  2.02s/it]

...

Epoch [8/10], Train Loss: 8.1222, Val Loss: 12.8958


 90%|█████████ | 9/10 [00:16<00:01,  1.64s/it]

Epoch [9/10], Train Loss: 4.5765, Val Loss: 12.9920


100%|██████████| 10/10 [00:17<00:00,  1.80s/it]

Epoch [10/10], Train Loss: 6.0617, Val Loss: 14.2406


In [8]:
torch.save(model.state_dict(), 'digit_sum_model.pth')